In [2]:
# Imports
import re
import time
import json
import requests
import bs4
from bs4 import BeautifulSoup as bs

In [ ]:
# URL
search = "https://www.linkedin.com/jobs/search/?currentJobId=3743834990&keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"

In [163]:
def getJobsURL(url_search: str , start_page: int = 0, num_pages: int = 1, max_jobs: int = 500):

    jobs_url = []
    c = 1
        
    for page in range(0, num_pages):

        job_page = num_pages * 25

        url = url_search + f'&start={job_page}'

        # Request
        rq = requests.get(url)
        if rq.status_code != 200:
            Exception('Invalid Request')

        #Parse do html
        soup = bs(rq.text, "html.parser")

        list_href = soup.select('.base-card__full-link')

        for i in range(len(list_href)):
            jobs_url.append(list_href[i]['href'])
            print(f'Collecting Job URL {c}/{num_pages * 25}')
            c += 1
            time.sleep(0.5)
            if c == max_jobs: break

    return jobs_url
    
jobs_url = getJobsURL(search)
#print(soup.prettify())   

In [164]:
# Iterate over the jobsURL
def getJobSoup(job_urls):
    soup_jobs = []
    c = 1

    for i in range(len(job_urls)): 
        url = job_urls[i]

        # Request
        rq = requests.get(url)
        if rq.status_code != 200:
            Exception('Invalid Request')

        #Parse do html
        soup = bs(rq.text, "html.parser")

        soup_jobs.append(soup)

        print(f'Collecting Job Post {c}/{num_pages * 25}')
        c += 1
        time.sleep(0.5)
    
    return soup_jobs

soup_jobs = getJobSoup(jobs_url)

In [166]:
def getJobAttr(soup_job):

    # String JSON
    json_string = soup_job.find_all('script', attrs = {'type':'application/ld+json'})[0].text

    # Convert a string JSON to Python Object
    json_obj = json.loads(json_string)

    # Now, the json is a Python dictionary
    return json_obj

getJobAttr(soup_jobs[0])

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2023-11-06T00:00:00.000Z',
 'description': '&lt;strong&gt;Description&lt;br&gt;&lt;br&gt;&lt;/strong&gt;&lt;strong&gt;About Us: &lt;br&gt;&lt;br&gt;&lt;/strong&gt;Cover Whale is making roads safer by merging insurance solutions with a data-driven driver safety program. Our straightforward insurance options help drivers navigate rising costs while providing solid results for our supporting insurers. We kickstarted our journey in 2019 with a smashing $15.5M seed funding round, the biggest InsurTech seed round in North America up-to-date, and proud to make Forbes\' list of "America\'s Best Startup Employers 2023". We\'re growing swiftly, and we\'d love you to join our journey.&lt;br&gt;&lt;br&gt;&lt;strong&gt;The Role: &lt;br&gt;&lt;br&gt;&lt;/strong&gt;In this role, you will be responsible for extracting, analyzing, and interpreting large datasets to drive insights for the company\'s programs and operations. The co

In [167]:
#### Metadados
def getJobMetaData(soup_job):
    postMetaData = {}

    postMetaData['jobURL'] = soup_job.find_all('meta', attrs = {'property':'og:url'})[0]['content']
    postMetaData['jobTitle'] = soup_job.find('title').text
    postMetaData['jobLocale'] = soup_job.find_all('meta', attrs = {'name':'locale'})[0]['content']
    postMetaData['jobTimePosted'] = ((soup_job.find_all('meta', attrs = {'name':'description'})[0]['content']).split('.')[0]).replace('Posted ', '')
    postMetaData['jobCompanyId'] = soup_job.find_all('meta', attrs = {'name':'companyId'})[0]['content']
    postMetaData['jobIndustryId'] = soup_job.find_all('meta', attrs = {'name':'industryIds'})[0]['content']
    # postMetaData['jobDescription'] = soup_job.find_all('meta', attrs = {'name':'description'})[0]['content']

    return postMetaData

getJobMetaData(soup_jobs[0])

{'jobURL': 'https://www.linkedin.com/jobs/view/data-analyst-i-at-cover-whale-3759423245',
 'jobTitle': 'Cover Whale hiring Data Analyst I in New York, New York, United States | LinkedIn',
 'jobLocale': 'en_US',
 'jobTimePosted': '12:00:00 AM',
 'jobCompanyId': '40902095',
 'jobIndustryId': '42,6,43'}

In [168]:
def getFullJobDescription(soup_job):

    boldItens = []

    for item in range(len(soup_job.select('.show-more-less-html__markup strong'))):
        boldItens.append(soup_job.select('.show-more-less-html__markup strong')[item].text)

    jobDescription = (soup_job.select('.show-more-less-html__markup')[0].text).replace('\n', ' ')

    fullDescription = {
                        'boldItens': boldItens,
                        'jobDescription': jobDescription
                        }
                        
    return fullDescription

getFullJobDescription(soup_jobs[0])

{'boldItens': ['Description',
  'About Us: ',
  'The Role: ',
  'Responsibilities: ',
  'Compensation:',
  'Requirements',
  'Education and Experience:',
  'Required Practical Skills: ',
  'Further Requirements and Professional Abilities: ',
  'Benefits',
  'Perks/Benefits:'],
 'jobDescription': ' DescriptionAbout Us: Cover Whale is making roads safer by merging insurance solutions with a data-driven driver safety program. Our straightforward insurance options help drivers navigate rising costs while providing solid results for our supporting insurers. We kickstarted our journey in 2019 with a smashing $15.5M seed funding round, the biggest InsurTech seed round in North America up-to-date, and proud to make Forbes\' list of "America\'s Best Startup Employers 2023". We\'re growing swiftly, and we\'d love you to join our journey.The Role: In this role, you will be responsible for extracting, analyzing, and interpreting large datasets to drive insights for the company\'s programs and oper

In [169]:
def getJobPost(soup_job):
    jobPost = {
        'attributes': getJobAttr(soup_job),
        'metadata': getJobMetaData(soup_job),
        'fullDescription': getFullJobDescription(soup_job)
    }
    return jobPost

getJobPost(soup_jobs[0])

{'attributes': {'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'datePosted': '2023-11-06T00:00:00.000Z',
  'description': '&lt;strong&gt;Description&lt;br&gt;&lt;br&gt;&lt;/strong&gt;&lt;strong&gt;About Us: &lt;br&gt;&lt;br&gt;&lt;/strong&gt;Cover Whale is making roads safer by merging insurance solutions with a data-driven driver safety program. Our straightforward insurance options help drivers navigate rising costs while providing solid results for our supporting insurers. We kickstarted our journey in 2019 with a smashing $15.5M seed funding round, the biggest InsurTech seed round in North America up-to-date, and proud to make Forbes\' list of "America\'s Best Startup Employers 2023". We\'re growing swiftly, and we\'d love you to join our journey.&lt;br&gt;&lt;br&gt;&lt;strong&gt;The Role: &lt;br&gt;&lt;br&gt;&lt;/strong&gt;In this role, you will be responsible for extracting, analyzing, and interpreting large datasets to drive insights for the company\'s programs and 

In [ ]:
### Full Script

# Imports
import re
import time
import json
import requests
from bs4 import BeautifulSoup as bs

search = "https://www.linkedin.com/jobs/search/?currentJobId=3743834990&keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"

jobs_url = getJobsURL(search, num_pages=1)

soup_jobs = getJobSoup(jobs_url)

jobs_posts = []
for job in range(len(soup_jobs)):
    jobs_posts.append(getJobPost(soup_jobs[job]))


